In [9]:
import rasterio

# URL to the COG file on AWS S3
cog_url = 'https://ccemp-bucket.s3.amazonaws.com/Public%20Access/COG/GTA-Halton-LidarDTM-A_cog.tif'

# Open the COG and print metadata
with rasterio.open(cog_url) as src:
    print("Raster Width:", src.width)
    print("Raster Height:", src.height)
    print("Raster CRS:", src.crs)
    print("Raster Bounds:", src.bounds)  # Geographical bounds of the raster


Raster Width: 46000
Raster Height: 46000
Raster CRS: EPSG:26917
Raster Bounds: BoundingBox(left=569000.0, bottom=4791000.0, right=592000.0, top=4814000.0)


In [20]:
import os
import sys
import json
import rasterio
from rasterio.windows import from_bounds
from rasterio.enums import Resampling

# URL to the COG file on AWS S3
cog_url = 'https://ccemp-bucket.s3.amazonaws.com/Public%20Access/COG/GTA-Halton-LidarDTM-A_cog.tif'

# Determine the default download folder based on the operating system
if sys.platform == "darwin":  # macOS
    download_folder = os.path.join(os.path.expanduser("~"), "Downloads")
elif sys.platform == "win32":  # Windows
    download_folder = os.path.join(os.path.expanduser("~"), "Downloads")
else:
    raise OSError("Unsupported operating system")

# Open the COG file and get its metadata
with rasterio.open(cog_url) as src:
    bounds = src.bounds
    transform = src.transform
    
    # Calculate the center of the raster and define a 5x5 meter block around it
    center_x, center_y = (bounds.left + bounds.right) / 2, (bounds.bottom + bounds.top) / 2
    block_size = 5000  # In meters
    x1, y1, x2, y2 = center_x - block_size / 2, center_y - block_size / 2, center_x + block_size / 2, center_y + block_size / 2

    x1 = x1 + 5000
    x2 = x2 + 5000

    # Print the calculated bounding box for the center zone
    print(f"Bounding box (5x5 meters) around the center: x1={x1}, y1={y1}, x2={x2}, y2={y2}")

    # Convert the geographic bounding box to a pixel window
    window = from_bounds(x1, y1, x2, y2, transform)
    
    # Adjust the metadata to match the window size
    meta = src.meta.copy()
    meta.update({
        "driver": "GTiff",
        "height": window.height,
        "width": window.width,
        "transform": src.window_transform(window),
        "count": 3,  # RGB has 3 bands
        "dtype": src.dtypes[0]  # Use the original data type to preserve quality
    })

    # Output file path (where the subset will be saved)
    output_file = os.path.join(download_folder, "center_5x5_rgb_highest_res.tif")
    
    # Write the window to a new GeoTIFF file
    with rasterio.open(output_file, 'w', **meta) as dst:
        for i in range(1, 4):  # RGB has 3 bands
            dst.write(src.read(i, window=window, resampling=Resampling.nearest), i)

print(f"Subset saved as {output_file}")

Bounding box (5x5 meters) around the center: x1=583000.0, y1=4800000.0, x2=588000.0, y2=4805000.0
Subset saved as /Users/shuyang/Downloads/center_5x5_rgb_highest_res.tif
